# Lab 9 CNNs & 2D Convolution – Coding Exercises



These exercises are based on the lecture slides about CNNs and 2D convolution.
Fill in the code where marked and use the checkup cells to verify your answers.

In [3]:
import numpy as np

SEED = 42
rng = np.random.default_rng(SEED)

np.set_printoptions(suppress=True)

def check_equal(actual, expected, tol=1e-8):
    """Simple helper to compare arrays / numbers.

    Prints a success message if the check passes, otherwise raises an AssertionError.
    """
    actual_arr = np.array(actual, dtype=float)
    expected_arr = np.array(expected, dtype=float)
    assert actual_arr.shape == expected_arr.shape, f"Shape mismatch: got {actual_arr.shape}, expected {expected_arr.shape}"
    assert np.allclose(actual_arr, expected_arr, atol=tol), f"Values differ. Max abs diff = {np.max(np.abs(actual_arr - expected_arr))}"
    print("✅ Check passed!")

## 1. Images as arrays

In the slides, grayscale images are represented as 2D arrays and RGB images as 3D tensors.
We'll warm up with a few simple array manipulations.

### Exercise 1 – Creating grayscale and RGB images (2P)
1. Create a 4×4 **grayscale** image `gray_img` with values from 0 to 150 in steps of 10 (row-wise).
2. Create a 4×4×3 **RGB** image `rgb_img` by stacking the grayscale image into 3 channels.
3. Verify the shapes using the checkup cell.


In [4]:
# TODO: create gray_img and rgb_img as described above.

gray_img = np.arange(0, 160, 10).reshape(4, 4)
rgb_img = np.stack([gray_img]*3, axis=-1)

gray_img, None if rgb_img is None else rgb_img.shape

(array([[  0,  10,  20,  30],
        [ 40,  50,  60,  70],
        [ 80,  90, 100, 110],
        [120, 130, 140, 150]]),
 (4, 4, 3))

In [5]:
# Checkup for Exercise 1
expected_gray = np.array([[0.0, 10.0, 20.0, 30.0], [40.0, 50.0, 60.0, 70.0], [80.0, 90.0, 100.0, 110.0], [120.0, 130.0, 140.0, 150.0]], dtype=float)
expected_rgb_shape = (4, 4, 3)

check_equal(gray_img, expected_gray)
assert rgb_img is not None, "rgb_img is still None"
assert rgb_img.shape == expected_rgb_shape, f"Expected shape {expected_rgb_shape}, got {rgb_img.shape}"
print("✅ rgb_img has the correct shape!")

✅ Check passed!
✅ rgb_img has the correct shape!


## 2. The 2D convolution 

In the slides, a single convolution step combines a small image region with a filter.
We'll implement this explicitly for a 2×2 region and 2×2 kernel.

### Exercise 2 – Single convolution step (2P)
Implement the function `conv2d_single_step(region, kernel)` that:
- assumes both `region` and `kernel` are 2×2 arrays,
- returns the scalar dot product `np.sum(region * kernel)`.


In [6]:
def conv2d_single_step(region, kernel):
    """Compute a single 2×2 convolution step.

    Parameters
    ----------
    region : array-like of shape (2, 2)
    kernel : array-like of shape (2, 2)
    """
    # TODO: replace the next line with your implementation
    region = np.array(region)
    kernel = np.array(kernel)
    return np.sum(region * kernel)


# Example inputs you can play with (optional)
region = np.array([[0., 10.],
                   [40., 50.]])
kernel = np.array([[1., 0.],
                   [0., -1.]])

In [7]:
# Checkup for Exercise 2
test_region = [[0.0, 10.0], [40.0, 50.0]]
test_kernel = [[1.0, 0.0], [0.0, -1.0]]

value = conv2d_single_step(test_region, test_kernel)
expected = -50.0
check_equal(value, expected)

✅ Check passed!


### Exercise 3 – Implement `conv2d_valid` (2P)
Now implement a function that performs a full **valid** 2D convolution:

`output[i, j] = sum_{m, n} image[i+m, j+n] * kernel[m, n]`

Assume:
- `image` is a 2D NumPy array of shape (H, W),
- `kernel` is a 2D NumPy array of shape (kH, kW),
- no padding and stride 1 (valid convolution).


In [8]:
def conv2d_valid(image, kernel):
    """Perform a valid 2D convolution (no padding, stride 1).

    Parameters
    ----------
    image : 2D NumPy array of shape (H, W)
    kernel : 2D NumPy array of shape (kH, kW)
    """
    image = np.array(image)
    kernel = np.array(kernel)

    H, W = image.shape
    kH, kW = kernel.shape

    # Output size for valid convolution
    out_H = H - kH + 1
    out_W = W - kW + 1

    output = np.zeros((out_H, out_W), dtype=float)

    for i in range(out_H):
        for j in range(out_W):
            region = image[i:i+kH, j:j+kW]
            output[i, j] = conv2d_single_step(region, kernel)

    return output


# After implementing, run conv2d_valid(gray_4x4, kernel_2x2) to see the result

In [9]:
# Checkup for Exercise 3
image = np.array([[0.0, 10.0, 20.0, 30.0], [40.0, 50.0, 60.0, 70.0], [80.0, 90.0, 100.0, 110.0], [120.0, 130.0, 140.0, 150.0]], dtype=float)
kernel = np.array([[1.0, 0.0], [0.0, -1.0]], dtype=float)

out = conv2d_valid(image, kernel)
expected = np.array([[-50.0, -50.0, -50.0], [-50.0, -50.0, -50.0], [-50.0, -50.0, -50.0]], dtype=float)
check_equal(out, expected)

✅ Check passed!


## 3. Sobel filters for edge detection
The slides introduce Sobel filters $S_x$ and $S_y$ to approximate horizontal and vertical gradients.
We'll recreate them and apply them to a small dummy image.

### Exercise 4 – Define Sobel filters (2P)
1. Define `sobel_x` (for vertical edges) and `sobel_y` (for horizontal edges) as 3×3 NumPy arrays.
2. Use the exact values from the slides.

In [10]:
# TODO: define sobel_x and sobel_y as described above.

sobel_x = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=float)

sobel_y = np.array([
    [-1, -2, -1],
    [ 0,  0,  0],
    [ 1,  2,  1]
], dtype=float)

### Exercise 5 – Apply Sobel filters (2P)
We use the following 6×6 dummy image (values 0 or 255) similar to the slides:
a vertical bright bar in the middle.

Steps:
1. Store the image in a variable `dummy_img`.
2. Use your `conv2d_valid` to compute `gx = conv2d_valid(dummy_img, sobel_x)` and `gy = conv2d_valid(dummy_img, sobel_y)`.
3. Compute the gradient magnitude `g = np.sqrt(gx**2 + gy**2)`.


In [11]:
# Step 1: store the dummy image
dummy_img = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 255.0, 255.0, 0.0, 0.0], [0.0, 0.0, 255.0, 255.0, 0.0, 0.0], [0.0, 0.0, 255.0, 255.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], dtype=float)

# Step 2 & 3: compute gx, gy and gradient magnitude g
# TODO: replace the lines below
gx = conv2d_valid(dummy_img, sobel_x)
gy = conv2d_valid(dummy_img, sobel_y)
g = np.sqrt(gx**2 + gy**2)

In [12]:
# Checkup for Exercise 5
expected_gx = np.array([[765.0, 765.0, -765.0, -765.0], [1020.0, 1020.0, -1020.0, -1020.0], [765.0, 765.0, -765.0, -765.0], [255.0, 255.0, -255.0, -255.0]], dtype=float)
expected_gy = np.array([[255.0, 765.0, 765.0, 255.0], [0.0, 0.0, 0.0, 0.0], [-255.0, -765.0, -765.0, -255.0], [-255.0, -765.0, -765.0, -255.0]], dtype=float)
expected_g = np.array([[806.3808033429367, 1081.8733752154178, 1081.8733752154178, 806.3808033429367], [1020.0, 1020.0, 1020.0, 1020.0], [806.3808033429367, 1081.8733752154178, 1081.8733752154178, 806.3808033429367], [360.62445840513925, 806.3808033429367, 806.3808033429367, 360.62445840513925]], dtype=float)

check_equal(gx, expected_gx)
check_equal(gy, expected_gy)
check_equal(g, expected_g)

✅ Check passed!
✅ Check passed!
✅ Check passed!
